# 功能合併

In [4]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
# ------------------------------------------------- modified for captcha
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# ------------------------------------------------- modified for captcha

def to_df():
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.find_all('tr')[1:]
    rows = list()
    for tr in trs:
        rows.append([td.text.replace('\n', '').replace('\xa0', '').replace('\t', '') for td in tr.find_all('td')])
    df = pd.DataFrame(data=rows)
    return df    

def grab_GPA(semester):
    # 選所有課程
    Select(driver.find_element(by=By.ID, value="qyt_id")).select_by_visible_text(semester)
    driver.find_element(by=By.NAME, value="Submit").click()

    # 按掉alert
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    driver.switch_to.alert.accept()

    # 等所有資料load
    sleep(10)
    
    # 表格轉dataframe
    df=to_df()
    df.columns=['科號 Course No.','科目名稱 Course Name','授課教師 Teacher','修課人數 Enrollment',
                '平均值 average GPA','標準差 standard deviation']
    #上一頁
    driver.find_element(by=By.NAME, value="Submit").click()
    
    return df
    
# ------------------------------------------------- modified for Brady's computer
options = Options()
options.add_argument("--disable-notifications")
options.add_experimental_option('excludeSwitches', ['enable-logging']) # shut up error code triggered by chrome: "Failed to read descriptor from node connection: A device attached to the system is not functioning." 

# win chrome
driver_path = './chromedriver_win'
s = Service(driver_path)
driver = webdriver.Chrome(service= s, options= options)
# ------------------------------------------------- modified for Brady's computer

# # ------ original
# driver = webdriver.Chrome()
# # ------ original
verificationErrors = []
accept_next_alert = True

driver.get("https://www.ccxp.nthu.edu.tw/ccxp/INQUIRE/")

driver.find_element(by=By.NAME, value="account").clear()
driver.find_element(by=By.NAME, value="account").send_keys("108060071") #要改成自己的帳號
driver.find_element(by=By.NAME, value="passwd").clear()
driver.find_element(by=By.NAME, value="passwd").send_keys("Hanna891210") #要改成自己的密碼
driver.find_element(by=By.XPATH, value=u"(.//*[normalize-space(text()) and normalize-space(.)='密 碼:'])[1]/following::tr[1]").click()
driver.find_element(by=By.NAME, value="passwd2").click()
driver.find_element(by=By.NAME, value="passwd2").clear()

# ------------------------------------------------- modified for captcha
from captcha import captcha_decoder
image_url = driver.find_element(By.XPATH, "//input[@name='passwd2']/following-sibling::img").get_attribute("src")
captcha_result = captcha_decoder(image_url)
captcha = driver.find_element(By.NAME, "passwd2")
captcha.send_keys(captcha_result)
# ------------------------------------------------- modified for captcha
# # ------ 手動輸入 驗證碼
# sleep(5)
# # ------ 手動輸入 驗證碼
driver.find_element(by=By.NAME, value="Submit").click()



In [ ]:
# 切換到修課同學
driver.switch_to.frame(1)
driver.find_element(by=By.ID, value="nodeIcon15").click() #課程、成績
driver.find_element(by=By.ID, value="itemIcon21").click() #修課同學查詢

# 切到中間的畫面
driver.switch_to.parent_frame()
driver.switch_to.frame(2)

GPA_110_1=grab_GPA("110上(2021-Fall)")
GPA_109_1=grab_GPA("109上(2020-Fall)") 

# 切到左邊側攔
driver.switch_to.parent_frame()
driver.switch_to.frame(1)
driver.find_element(by=By.ID, value="nodeIcon54").click() # 選課
driver.find_element(by=By.ID, value="nodeIcon61").click() # 報表
driver.find_element(by=By.ID, value="itemIcon62").click() # 選上/剩餘名額/待亂數人數統計

# 切到中間的畫面
driver.switch_to.parent_frame()
driver.switch_to.frame(2)

# 選通識中心課程
Select(driver.find_element(by=By.NAME, value="select")).select_by_visible_text('GE 通識中心')
driver.find_element(by=By.NAME, value="Submit").click()

# 等資料load
sleep(5)

# 表格轉dataframe
enrollment_status_df=to_df()
enrollment_status_df.columns=['科號 Course Number.','科目名稱 Course Name','教師 Professor','通識類別',
                              '時間 Class Time','人數限制 / 新生保留 Size Limit / Freshman only','目前選上人數 Current Number',
                              '目前剩餘名額 Current Spots Remaining','目前待亂數人數 To be randomed']
enrollment_status_df=enrollment_status_df.drop([0])
enrollment_status_df=enrollment_status_df.drop([1])

driver.quit()

In [55]:
GPA_110_1

,科號 Course No.,科目名稱 Course Name,授課教師 Teacher,修課人數 Enrollment,平均值 average GPA,標準差 standard deviation
0,11010AES 450600,應用生態學Applied Ecology,"楊樹森YANG, SHU-SEN",9,3.89,0.3
1,11010AES 510000,專題演講Colloquium,"周子勤CHOU, TSU-CHIN",40,3.84,0.19
2,11010AES 510100,碩士班專題研究MS Graduate Research,指導教授,15,4.19,0.26
3,11010AES 511100,書報討論Seminar,"周子勤CHOU, TSU-CHIN",22,3.85,0.32
4,11010AES 520100,環境分析化學Environmental Analytical Chemistry,"黃郁棻HUANG, YU-FEN",20,3.65,0.5
...,...,...,...,...,...,...
2972,11010ZY 100240,服務學習--幼教系Student Service,幼教系,43,3.97,0.63
2973,11010ZY 100242,服務學習--教科系Student Service,教科系,41,4.3,0
2974,11010ZY 100244,服務學習--教育院學士班Student Service,教育院學士班,16,3.38,1.16
2975,11010ZY 100245,服務學習--藝術學院學士班Student Service,藝術院學士班,31,3.87,1.11


In [56]:
enrollment_status_df

,科號 Course Number.,科目名稱 Course Name,教師 Professor,通識類別,時間 Class Time,人數限制 / 新生保留 Size Limit / Freshman only,目前選上人數 Current Number,目前剩餘名額 Current Spots Remaining,目前待亂數人數 To be randomed
2,11110HIS 233700,歷史思維：知人論世(增開)Historical Thinking,"李卓穎LI, CHO-YING",向度1,M3M4,20,0,20,4
3,11110HIS 233100,歷史思維：歷史與思考How to learn and think history?,"許松源SHU, SUNG-YUAN",向度1,W3W4,60,20,40,4
4,11110GEC 170401,歷史現場：歷史文獻、現地考察與歷史書寫Historical Fieldwork: Histo...,"張繼瑩CHANG, CHI-YING",向度1,R7R8R9,50/5,25,25,3
5,11110GEC 170202,歷史與世界：11世紀以降History and the World: Since the 1...,陳麗華CHEN LI HUA,向度1,T3T4Tn,60/6,35,25,7
6,11110GEC 112100,思想經典：民國儒家思想The Philosophy of Confucianism in M...,"楊儒賓YANG, RUR-BIN",向度1,R7R8R9,60/6,36,24,12
...,...,...,...,...,...,...,...,...,...
238,11110GE 131800,現代詩選讀Chinese Modern Poetry,"陳柏伶CHEN, PO-LING",人文,M5M6,50,30,20,10
239,11110HIS 260500,德國史The German History,"馬文英MA, WEN-YING",人文,W3W4,80,33,47,11
240,11110GE 115800,台灣當代戲曲Taiwan Contemporary Xiqu,"吳岳霖WU, YUE-LIN",人文,M5M6,55,54,1,7
241,11110GE 132800,"中國神話故事選讀Selective Readings for ""The Chinese My...","陳燕梅CHEN, YEN-MEI",人文,F7F8,55,54,1,11


# 全校GPA

# 選上/剩餘名額/待亂數人數統計

## Checkbox

In [57]:
DAY_NUM = 6
TIME_NUM = 13
CLASS_NUM = len(enrollment_status_df.index)
day_dict = {"M":0,'T':1,"W":2,'R':3,'F':4,'S':5}
time_dict = {"1":0,"2":1,"3":2,"4":3,"n":4,"5":5,"6":6,"7":7,"8":8,"9":9,"a":10,"b":11,"c":12}
inv_time_dict = {v: k for k, v in time_dict.items()}
# Import Tkinter library
from tkinter import *
# Create an instance of tkinter frame
win = Tk()
win.title("請勾選欲篩選的時段")

# Set the geometry of Tkinter frame
win.geometry("720x540")
# Define empty variables
days = ['M','T','W','R','F','S']
slots = [
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()],
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()],
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()],
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()],
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()],
[BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar(),BooleanVar()]
]
# Define a Checkbox
for i in range(DAY_NUM):
    for j in range(TIME_NUM):
        cb = Checkbutton(win, text=days[i]+inv_time_dict[j], variable=slots[i][j], onvalue=1, offvalue=0,cursor="heart")
        cb.grid(row=j+2, column=i)
       

win.mainloop()

## filter by time

In [58]:
def filter_time(row):
    s = row['時間 Class Time']
    if(s==' '):
        return True
    for i in range(0,len(s),2):        
        d = day_dict[s[i]]
        t = time_dict[s[i+1]]
        if(not slots[d][t].get()):
            return False
    return True


filtered_df = pd.DataFrame(enrollment_status_df)
filtered_df= filtered_df[filtered_df.apply(filter_time, axis=1)]

## 選上機率

In [59]:
filtered_df['目前剩餘名額 Current Spots Remaining'] = pd.to_numeric(filtered_df['目前剩餘名額 Current Spots Remaining'])
filtered_df['目前待亂數人數 To be randomed'] = pd.to_numeric(filtered_df['目前待亂數人數 To be randomed'])

c1 =[]
for index, row in filtered_df.iterrows():
    if(row['目前剩餘名額 Current Spots Remaining'] == 0):
        c1.append(0)
    elif(row['目前剩餘名額 Current Spots Remaining'] != 0 and row['目前待亂數人數 To be randomed'] == 0):
        c1.append(100)
    elif(row['目前剩餘名額 Current Spots Remaining'] > row['目前待亂數人數 To be randomed']):
        c1.append(100)
    else:
        c1.append((row['目前剩餘名額 Current Spots Remaining'] / row['目前待亂數人數 To be randomed'])*100)

filtered_df['選上機率'] = c1

## GPA

In [60]:
c2 =[]
c2_neg =[]
c2_pos =[]
for index_1,row_1 in filtered_df.iterrows():
    flag_name = 0
    for index_2,row_2 in GPA_110_1.iterrows():
        if (row_1['科目名稱 Course Name'] == row_2['科目名稱 Course Name']):
            c2.append(row_2['平均值 average GPA'])
            c2_neg.append(float(row_2['平均值 average GPA']) - 2* float(row_2['標準差 standard deviation']))
            c2_pos.append(float(row_2['平均值 average GPA']) + 2* float(row_2['標準差 standard deviation']))
            flag_name = 1
            break
    flag_no = 0
    if(flag_name == 0):
        for index_2,row_2 in GPA_110_1.iterrows():
            if (row_1['科號 Course Number.'][-6:] == row_2['科號 Course No.'][-6:]):
                c2.append(row_2['平均值 average GPA'])
                c2_neg.append(float(row_2['平均值 average GPA']) - 2* float(row_2['標準差 standard deviation']))
                c2_pos.append(float(row_2['平均值 average GPA']) + 2* float(row_2['標準差 standard deviation']))
                flag_no = 1
                break
        if(flag_no == 0):
                c2.append(-1)
                c2_neg.append(-1)
                c2_pos.append(-1)

filtered_df ['110上GPA'] = c2
filtered_df ['110上GPA-兩倍標準差'] = c2_neg
filtered_df ['110上GPA+兩倍標準差'] = c2_pos

In [61]:
c3 =[]
c3_neg =[]
c3_pos =[]
for index_1,row_1 in filtered_df.iterrows():
    flag_name = 0
    for index_2,row_2 in GPA_109_1.iterrows():
        if (row_1['科目名稱 Course Name'] == row_2['科目名稱 Course Name']):
            c3.append(row_2['平均值 average GPA'])
            c3_neg.append(float(row_2['平均值 average GPA']) - 2* float(row_2['標準差 standard deviation']))
            c3_pos.append(float(row_2['平均值 average GPA']) + 2* float(row_2['標準差 standard deviation']))
            flag_name = 1
            break
    flag_no = 0
    if(flag_name == 0):
        for index_2,row_2 in GPA_109_1.iterrows():
            if (row_1['科號 Course Number.'][-6:] == row_2['科號 Course No.'][-6:]):
                c3.append(row_2['平均值 average GPA'])
                c3_neg.append(float(row_2['平均值 average GPA']) - 2* float(row_2['標準差 standard deviation']))
                c3_pos.append(float(row_2['平均值 average GPA']) + 2* float(row_2['標準差 standard deviation']))
                flag_no = 1
                break
        if(flag_no == 0):
                c3.append(-1)
                c3_neg.append(-1)
                c3_pos.append(-1)

filtered_df ['109上GPA'] = c3
filtered_df ['109上GPA-兩倍標準差'] = c3_neg
filtered_df ['109上GPA+兩倍標準差'] = c3_pos

## sorting

In [62]:
filtered_df_sort= filtered_df.sort_values(['選上機率', '110上GPA','109上GPA'], ascending = [False, False, False])

In [63]:
filtered_df_sort

,科號 Course Number.,科目名稱 Course Name,教師 Professor,通識類別,時間 Class Time,人數限制 / 新生保留 Size Limit / Freshman only,目前選上人數 Current Number,目前剩餘名額 Current Spots Remaining,目前待亂數人數 To be randomed,選上機率,110上GPA,110上GPA-兩倍標準差,110上GPA+兩倍標準差,109上GPA,109上GPA-兩倍標準差,109上GPA+兩倍標準差
133,11110GE 152400,印度文化概論Introduction to Indian Culture,"董玉莉TUNG, YU-LI",社會,R3R4,50,7,43,3,100.000000,3.7,2.58,4.82,3.65,2.75,4.55
156,11110GE 199400,新聞學概論Introduction to Journalism,"蒯亮LIANG, KUAI",社會,R1R2,50,37,13,10,100.000000,3.62,2.16,5.08,3.19,1.21,5.17
7,11110HIS 213100,歷史文化經典：中國上古歷史人物淺談An Analysis of Chinese Histor...,"許松源SHU, SUNG-YUAN",向度1,R3R4,60,35,25,8,100.000000,2.87,0.77,4.97,3.26,1.08,5.44
161,11110GE 135500,高科技創業入門Introduction to High-Tech Entrepreneurship,"蔡仁松TSAY, REN-SONG",社會,,300,50,250,18,100.000000,-1,-1.00,-1.00,-1,-1.00,-1.00
20,11110GEC 110302,價值與實踐Value and Practice,"洪巳軒HUNG, SZU-HSUAN",向度1,R2R3R4,70/7,61,9,11,81.818182,3.48,1.90,5.06,3.36,1.68,5.04
159,11110GE 134500,廣告學概論Introduction to Advertising,"蒯亮LIANG, KUAI",社會,R3R4,50,46,4,7,57.142857,3.9,3.08,4.72,2.86,0.60,5.12
108,11110GE 107200,中藥與近代科技Chinese Herbology and Modern Sciences &...,"張翔CHANG, SHYANG陳瑞凱CHEN, SWE-KAI",自然,R2R3R4,60,58,2,7,28.571429,3.52,2.04,5.00,3.71,1.97,5.45
150,11110GE 144400,愛情關係的發展與經營Development and management of love r...,"楊雅婷YANG, YA-TING",社會,R3R4,50,49,1,13,7.692308,3.7,2.54,4.86,3.62,1.78,5.46
